# Implémentation et test du modèle XGBOOST
---

Dans ce notebook, on va découvrir les différents aspects du modèle XGBOOST. En premier lieu nous allons implémenter le modèle à zéro (from scratch) ensuite en deuxième partie nous allons directement utiliser le modèle depuis l'officielle bibliothèque ([xgboost](https://github.com/dmlc/xgboost)) et effectuer avec différents tests.

In [6]:
# Importation des modules nécessaires

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.__version__, pd.__version__

('1.24.3', '2.0.1')

## Introduction

Pour bien débuter, nous posons d'abord les premières notions qu'on va utiliser directement dans les prochaines sections.
Tout au long du notebook, en parlant du dataset donné on note le nombre d'individus $n$ et le nombre de caractéristiques de chaque individu $m$. Ou plus explicitement le dataset ($D$) est donné comme il suit:
$$D = \{(x_i,y_i) \text{ tq } x_i \in R^m, y_i \in R\}$$

## I. Réalisation des algorithmes

Cette partie servira à la compréhension des algorithmes utilisés dans un modèle XGBOOST en les implémentant de zéro pour les deux cas (Régression et Classification). Pour cela nous allons utiliser seulement la bibliothèque **numpy** qui est utile dans les calculs matriciels.

### I.1. Fonction de prédiction

Pour un dataset donnée ($D$), la fonction de prédiction du modèle est la somme des $K + 1$ plus simples fonctions de prédiction. Les fonctions de prédiction ($f_k$) avec $k >= 1$ donne les résultats du parcours des arbres de régression construits lors de l'entrainement du modèle. Pour la fonction initiale ($f_0$), ce n'est qu'une valeur constante initiale déterminée au début de l'entraînement. On donne la fonction de prédiction du modèle donc comme il suit:
$$\hat{y_i} = \phi(x_i) = f_0 + \eta \sum\limits_{k = 1}\limits^{K} f_k(x_i)$$
Note: $\eta$ représente le taux d'apprentissage (learning rate)

In [7]:
def pred(X, f0, fs, eta=0.3):
    return f0 + eta * np.sum(np.array([fk(X) for fk in fs]), axis=0)

# Ceci n'est qu'un exemple d'une fonction d'un arbre pour faire fonctionner le test
def f_test(X):
    return np.sum(np.square(X) + X + 4, axis=1)

X = np.array(
    [
        [1, 3],
        [4, 5]
    ]
)

f0 = 3.4

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([16.6, 38.2])
#---------------------------------------------------------------------

pred(X, f0, [f_test, f_test])

array([16.6, 38.2])

### I.2 Fonction du coût

L'étape suivante serait de définir la fonction du coût (loss function) pour évaluer le modèle et le faire converger vers le point optimal. Or on a deux cas: la régression et la classification. Et donc on définira pour chacun des cas sa fonction de coût ensuite leurs dérivées pour pouvoir appliquer le [gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) par la suite. 

#### I.2.1 Fonction du coût (Régression)

Pour la régression, nous allons utiliser la somme des erreurs carrées (sum of squares error, SSE). Elle est définie comme il suit
$$SSE = l(y_i,\hat{y_i}) = \frac{1}{2} (y_i - \hat{y_i})^2$$
Note: la fraction $\frac{1}{2}$ est ajoutée juste pour simplifier les calculs (notamment pour la dérivée)

In [8]:
def SSE(Y, Y_pred):
    return (1/2) * np.square(Y - Y_pred)

Y = np.array([1.2, 3.5, 6.7])
Y_pred = np.array([1.0, 4, 7.2])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([0.02 , 0.125, 0.125])
#---------------------------------------------------------------------

SSE(Y, Y_pred)

array([0.02 , 0.125, 0.125])

Ensuite on calcule la dérivée de cette fonction de coût par rapport aux prédictions réalisées. Cela nous permettra de trouver la prédiction qui minimise la fonction coût. La dérivée est donc donnée comme il suit:
$$\frac{\partial SSE}{\partial \hat{y_i}} = \hat{y_i} - y_i$$
Note: La valeur $y_i - \hat{y_i}$ est appelée le résidu **$r_{ik}$** où $k$ représente le numéro de l'itération. Donc on a:
$$r_{ik} = - \frac{\partial SSE}{\partial \hat{y_i}}$$

In [9]:
def dSSE(Y, Y_pred):
    return Y_pred - Y

Y = np.array([1.2, 3.5, 6.7])
Y_pred = np.array([1.0, 4, 7.2])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([-0.2,  0.5,  0.5])
#---------------------------------------------------------------------

dSSE(Y, Y_pred)

array([-0.2,  0.5,  0.5])

#### I.2.2 Fonction du coût (Classification)

Pour la classification (dans ce cas binaire), nous allons utiliser l'inverse du log de vraisemblance (negative log-likelihood). Elle est définie comme il suit
$$NLL = l(y_i,p_i) = -\text{ }(\text{ }y_i \log(p_i) + (1-y_i) \log(1-p_i)\text{ })$$
Note: $p_i$ est la probabilité prédite par le modèle. Une valeur proche de 0 signifie que le modèle classe l'individu à la classe négative et une valeur proche de 1 signifie que le modèle classe l'individu à la classe positive.

In [13]:
def NLL(Y, Y_pred):
    return - (Y * np.log(Y_pred) + (1 - Y) * np.log(1 - Y_pred))

Y = np.array([1, 0, 1])
Y_pred = np.array([0.7, 0.2, 0.8])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([0.35667494, 0.22314355, 0.22314355])
#---------------------------------------------------------------------

NLL(Y, Y_pred)

array([0.35667494, 0.22314355, 0.22314355])

Ensuite on calcule la dérivée de cette fonction de coût par rapport au log de chance de gain (log of odds). Cela nous permettra de trouver la prédiction qui minimise la fonction coût. On rappele que la chance de gain (odds) d'un évènement de propababilité $p$ est définie par:
$$odds = \frac{p}{1 - p}$$
$$\log(odds) = \log(\frac{p}{1 - p})$$
Dériver par rapport au $\log(odds)$ est aussi un choix minutieux pour réduire et faciliter les calculs pour le modèle. Il est permis de faire ce choix car il y a une relation directe entre le $\log(odds)$ et la probabilité $p$. En effet, On peut récupérer la probalité $p$ depuis $\log(odds)$ ainsi:
$$p = \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}$$
Calculons maintenant la dérivée par rapport au $\log(odds)$:
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -(\text{ }y_i \log(p_i) + (1-y_i) \log(1-p_i)\text{ })$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(p_i) + y_i \log(1-p_i) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i (\log(p_i) - \log(1-p_i)) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(\frac{p_i}{1-p_i}) - \log(1-p_i)$$
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(odds) - \log(1-p_i)$$
On a:
$$\log(1-p_i) = \log(1 - \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}) = \log(\frac{1 + e^{\log(odds)}}{1 + e^{\log(odds)}} - \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}) = \log(\frac{1}{1 + e^{\log(odds)}})$$
$$\log(1-p_i) = -\log(1 + e^{\log(odds)})$$
Et donc:
$$\frac{\partial NLL}{\partial \log(odds)} = \frac{\partial}{\partial \log(odds)} -y_i \log(odds) + \log(1 + e^{\log(odds)})$$
$$\frac{\partial NLL}{\partial \log(odds)} = -y_i + \frac{e^{\log(odds)}}{1 + e^{\log(odds)}}$$
$$\frac{\partial NLL}{\partial \log(odds)} = p_i - y_i$$
Note: La valeur $y_i - p_i$ est aussi appelée le résidu $r_{ik}$ où $k$ représente le numéro de l'itération. Donc on a:
$$r_{ik} = - \frac{\partial NLL}{\partial \log(odds)}$$

In [14]:
def dNLL(Y, Y_pred):
    return Y_pred - Y

Y = np.array([1, 0, 1])
Y_pred = np.array([0.7, 0.2, 0.8])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# array([-0.3,  0.2, -0.2])
#---------------------------------------------------------------------

dNLL(Y, Y_pred)

array([-0.3,  0.2, -0.2])

### I.3 Constuction des arbres uniques de régression (Arbres XGBOOST)

La fonction de prédiction du modèle XGBOOST dépend des arbres de régression construit lors de l'entrainement. Ces arbres sont des arbres spéciaux appelés des arbres uniques (ou arbres XGBOOST) qui servent à prédir des résidus ($r_{ik}$). En additionnant leurs résultats multipliés par un taux d'apprentissage et ajouté à la prédiction initiale, cela nous donne la prédiction finale du modèle (gradient tree boosting).

#### I.3.1 Fonction de coût de l'arbre

Pendant la construction l'arbre, il faudrait minimiser une certaine fonction de coût (loss function). Son équation est donnée ci-dessous:
$$L^{(k)} = \sum\limits_{i=1}\limits^{n} l(y_i,\hat{y_i}^{(k-1)}+f_k(x_i)) + \Omega(f_k)$$
où
- $y_i$ est la sortie réelle pour l'individu $i$
- $\hat{y_i}^{(k)}$ est la prédiction pour l'individu $i$ faite par le modèle à l'itération **$k$**
- $f_k$ est la fonction qui donne pour tout individu $x_i$ le résultat de l'arbre construit à l'itération $k$
- $l(y_i,\hat{y_i})$ est la fonction de coût (SSE ou NLL dans notre cas)
- $\Omega(f_k)$ est un terme de régulation qui pénalise la complexité du modèle. Il est donné comme il suit:
$$\Omega(f_k) = \gamma T + \frac{1}{2} \lambda \sum\limits_{j=1}\limits^{T} w_j^2$$
où
- $T$ est le nombre de feuilles dans l'arbre
- $w_j$ est le poids de la feuille $j$ dans l'arbre (le calcul de ce poids sera détaillé juste après)
- $\gamma$ est un hyper-paramètre de régularisation, il définit la réduction minimale du coût pour réaliser une partition supplémentaire sur une feuille de l'arbre. Par défaut à 0, une plus grande valeur rend le modèle plus conservatif.
- $\lambda$ est un hyper-paramètre de régularisation L2 sur les poids des feuilles. Par défaut à 1, une plus grande valeur rend le modèle plus conservatif.

In [23]:
# Fonction de coût sans le terme de régularisation
def Lk_sans_reg(X, Y, Y_pred_past, fk, loss=SSE):
    return np.sum(loss(Y, Y_pred_past + fk(X)))

# Terme de régularisation
def reg(W, reg_gamma=0, T=32, reg_lambda=1):
    return reg_gamma * T + (1/2) * reg_lambda * np.sum(np.square(W))

# Fonction de coût complète
def Lk(X, Y, Y_pred_past, fk, W, loss=SSE, reg_gamma=0, T=32, reg_lambda=1):
    return Lk_sans_reg(X, Y, Y_pred_past, fk, loss) + reg(W, reg_gamma, T, reg_lambda)

X = np.array(
    [
        [1, 3],
        [4, 5]
    ]
)

Y = np.array([2.2, 8.7])
Y_pred_past = np.array([1.0, 7.2])

# Ceci n'est qu'un exemple d'une fonction d'un arbre pour faire fonctionner le test
def f_test(X):
    return np.sum(X/5)

W = np.array([0.2, 0.3, -0.8, 0.7])

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (1.5850000000000013, 1.03, 2.615000000000001)
#---------------------------------------------------------------------

Lk_sans_reg(X, Y, Y_pred_past, f_test, loss=SSE), reg(W, reg_gamma=0.1, T=4, reg_lambda=1), Lk(X, Y, Y_pred_past, f_test, W, loss=SSE, reg_gamma=0.1, T=4, reg_lambda=1)

(1.5850000000000013, 1.03, 2.615000000000001)

#### I.3.2 Approximation de la fonction coût de l'arbre

Maintenant que la fonction de coût est bien définie, nous pouvons trouver chaque sortie optimale $f_k(x_i)$ qui minimise le plus la fonction de coût. Celà nécessitera le calcul de dérivée de la fonction coût. Pour optimiser les calculs, le modèle XGBOOST approche le fonction coût par son polynôme de Taylor de second degré:
$$l(y_i,\hat{y_i}^{(k-1)}+f_k(x_i)) \approx l(y_i,\hat{y_i}^{(k-1)}) + \frac{\partial l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)}} f_k(x_i) + \frac{1}{2} \frac{\partial^2 l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)2}} f_k(x_i)^2$$
Et donc
$$L^{(k)} \approx \sum\limits_{i=1}\limits^{n} [\text{ } l(y_i,\hat{y_i}^{(k-1)}) + \frac{\partial l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)}} f_k(x_i) + \frac{1}{2} \frac{\partial^2 l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)2}} f_k(x_i)^2\text{ }] + \Omega(f_k)$$
$$L^{(k)} \approx \sum\limits_{i=1}\limits^{n} [\text{ } l(y_i,\hat{y_i}^{(k-1)}) + g_i f_k(x_i) + \frac{1}{2}h_i f_k(x_i)^2\text{ }] + \Omega(f_k)$$
où
- $g_i$ est la dérivée de la fonction coût par rapport à la prédiction précédente ($\frac{\partial l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)}}$)
- $h_i$ est la seconde dérivée de la fonction coût par rapport à la prédiction précédente ($\frac{\partial^2 l(y_i,\hat{y_i}^{(k-1)})}{\partial \hat{y_i}^{(k-1)2}}$)

#### I.3.3 Quality Score et Poids des feuilles
Définisons plus explicitement la fonction $f_k$, cette fonction nous donne le résultat renvoyé par l'arbre pour un individu avec ses caractéristiques. Ce résultat sera égal au poids d'une des feuilles ($w_j$) de la façon suivante:
$$\hat{y_i}^{(k)} = \hat{y_i}^{(k-1)} + f_k(x_i)$$
$$r_{ik} = y_i - \hat{y_i}^{(k)}$$
$$\text{Si } r_{ik} \in I_j \text{ tel que } I_j \text{ est l'ensemble des résidus se trouvant dans la feuille } j$$
$$\text{Donc } f_k(x_i) = w_j$$
où $w_j$ est le poids de la feuille $j$

On peut considérer maintenant une nouvelle fonction de coût approximative pour l'arbre, où on peut omettre le terme $l(y_i,\hat{y_i}^{(k-1)})$ qui ne représente que le coût de la précédente prédiction qui n'a pas besoin d'être minimisé dans la courante itération. Donc on a maintenant:

$$\tilde{L}^{(k)} = \sum\limits_{i=1}\limits^{n} [\text{ } g_i f_k(x_i) + \frac{1}{2}h_i f_k(x_i)^2\text{ }] + \gamma T + \frac{1}{2} \lambda \sum\limits_{j=1}\limits^{T} w_j^2$$
$$\tilde{L}^{(k)} = \sum\limits_{j=1}\limits^{T} [\text{ } (\sum\limits_{r_{ik} \in I_j}g_i) w_j + \frac{1}{2}(\sum\limits_{r_{ik} \in I_j}h_i + \lambda) w_j^2\text{ }] + \gamma T$$
En résolvant l'équation $\frac{\partial \tilde{L}^{(k)}}{\partial w_j} = 0$ On obtient:
$$w_j^* = -\frac{\sum\limits_{r_{ik} \in I_j}g_i}{\sum\limits_{r_{ik} \in I_j}h_i + \lambda}$$
$$\tilde{L}^{(k)}(w_j^*) = -\frac{1}{2} \sum\limits_{j=1}\limits^{T} \frac{(\sum\limits_{r_{ik} \in I_j}g_i)^2}{\sum\limits_{r_{ik} \in I_j}h_i + \lambda} + \gamma T$$
$$Q_j = - \tilde{L}^{(k)}(w_j^*)$$
où
- $w_j^*$ est le poids optimal de la feuille
- $\tilde{L}^{(k)}(w_j^*)$ est le coût minimal pour l'arbre
- $Q_j$ est appelée score de qualité ou similarité (Quality or Similarity Score). Puisque elle est inversement proportionelle au coût, le but est de la maximiser.

In [30]:
# On a maintenant toutes les informations pour créer un noeud de l'arbre
# On définit une classe abstraite pour représenter un noeud en général
class Noeud:

    nbr = 0
    indent = '\t'

    def __init__(self, num: int, quality_score: float, profondeur: int):
        self.num = num # le numéro du caractéristique de division dans X
        self.qs = quality_score # le quality score pour le noeud
        self.pr = profondeur # la profondeur du noeud
        self.fils = {} # les fils du noeud
        self.res = [] # les résidus trouvés dans le noeud (applicable que pour les feuilles)
        self.gs = [] # les dérivés gi de premier ordre du noeud (applicable que pour les feuilles)
        self.hs = [] # les dérivés hi de second ordre du noeud (applicable que pour les feuilles)
        self.w = 0 # poids du noeud (applicable que pour les feuilles)

    # Cette fonction est pour transformer le noeud à un string
    def __str__(self): ...

    # predire un échantillon
    def predire(self, x: list): ...

    # générer un code pour graphviz
    def graphviz(self): ...

# On définit un noeud qu'on appelle NoeudCat pour gérer les données qualitatives, cette classe héritera de la classe Noeud
class NoeudCat(Noeud):
    # Cette fonction est pour transformer le noeud à un string
    def __str__(self):
        tmp_indent = Noeud.indent * self.pr # indentation : esthetique
        
        # s'il n'y a pas de fils, le noeud est terminal ; on imprime la classe
        if (len(self.fils)==0):
            return tmp_indent + f'W = {self.w}\n'
        
        # s'il y a des fils, on boucle sur les fils et on imprime des SI ... ALORS
        res = ""
        for valeur in self.fils:
            res += tmp_indent + 'Si X[' + str(self.num) + '] est "' + str(valeur) 
            res += '" Alors\n' + str(self.fils[valeur])
        return res

    # predire un échantillon
    def predire(self, x: list[str]):
        # Si le noeud est final, il rend son poids
        if (len(self.fils)==0):
            return self.w
        
        # Si la valeur de la colonne respective à ce noeud n'appartient pas à l'ensemble des
        # valeurs attendues, on rend np.nan
        if not(x[self.num] in self.fils.keys()): 
            return np.nan
        
        # Sinon, on rend 
        return self.fils[x[self.num]].predire(x)

    # générer un code pour graphviz
    def graphviz(self):
        nid = 'N' + str(Noeud.nbr)
        Noeud.nbr += 1
        
        # Si le noeud est final, 
        if (len(self.fils)==0):
            return nid, nid + '[label="' + str(self.w) + '" shape=ellipse];\n'
        
        # Sinon, 
        # s'il y a des fils, on boucle sur les fils et on imprime des SI ... ALORS
        res  = nid + '[label="X[' + str(self.num) + ']\\n'
        res += 'QS = ' + str(self.qs) + '"];\n'
        for valeur in self.fils:
            vid, code = self.fils[valeur].graphviz()
            res += code
            res += nid + ' -> ' + vid + ' [label="' + valeur + '"];\n'
        return nid, res
    
# On définit un noeud qu'on appelle NoeudNum pour gérer les données quantitatives, cette classe héritera de la classe Noeud
class NoeudNum(Noeud):
    def __init__(self, num: int, quality_score: float, profondeur: int, val: float):
        self.num = num # le numéro du caractéristique de division dans X
        self.qs = quality_score # le quality score pour le noeud
        self.pr = profondeur # la profondeur du noeud
        self.val = val # la valeur du noeud
        self.fils = {} # les fils du noeud
        self.res = [] # les résidus trouvés dans le noeud (applicable que pour les feuilles)
        self.gs = [] # les dérivés gi de premier ordre du noeud (applicable que pour les feuilles)
        self.hs = [] # les dérivés hi de second ordre du noeud (applicable que pour les feuilles)
        self.w = 0 # poids du noeud (applicable que pour les feuilles)

    # Cette fonction est pour transformer le noeud à un string
    def __str__(self):

        tmp_indent = self.indent * self.pr # indentation : esthetique
        
        # s'il n'y a pas de fils, le noeud est terminal ; on imprime la classe
        if (len(self.fils)==0):
            return tmp_indent + f'W = {self.w}\n'
         
        prefix = ' < '
        suffix = ''
        
        # s'il y a des fils, on boucle sur les fils et on imprime des SI ... ALORS SINON
        res  = ''
        res += tmp_indent + 'Si X[' + str(self.num) + '] ' + prefix + str(self.val) + suffix 
        res += ' Alors\n' + str(self.fils['g'])
        res += tmp_indent + 'Sinon\n' + str(self.fils['d'])
        return res

    # predire un échantillon
    def predire(self, x: list):
        # Si le noeud est final, il rend son poids
        if (len(self.fils)==0):
            return self.w
        
        # sinon
        if x[self.num] < self.val:
            return self.fils['g'].predire(x)
        return self.fils['d'].predire(x)

    # générer un code pour graphviz
    def graphviz(self):
        nid = 'N' + str(Noeud.nbr)
        Noeud.nbr += 1
        
        # Si le noeud est final, 
        if (len(self.fils)==0):
            return nid, nid + '[label="' + str(self.w) + '" shape=ellipse];\n'
        
        # Sinon, 
        # s'il y a des fils, on boucle sur les fils et on imprime des SI ... ALORS
        prefix = '] < '
        res = nid + '[label="X[' + str(self.num) + prefix + str(self.val) + '\\n'
        res += 'QS = ' + str(self.qs) + '"];\n'
        vid_G, code_G = self.fils['g'].graphviz()
        vid_D, code_D = self.fils['d'].graphviz()
        
        res += code_G + code_D
        res += nid + ' -> ' + vid_G + ' [label="Vrai"];\n'
        res += nid + ' -> ' + vid_D + ' [label="Faux"];\n'
        return nid, res